# Geostatistics

## 4. Spatial Structure

With this lecture we will start to look at *distance* and *spatial* relations of observations. We will cover

* the concept of *distance*
* talk (again) about covariance
* introduce (co)variograms

**Disclaimer**: The code examples shown in this Notebook are **not** performance optimized. There are **way** better techniques to implement the shown examples in Python. The main focus is on the underlying **theory**, not the code!

In [1]:
import base64
from pprint import pprint
from io import BytesIO
from collections import Counter

import numpy as np
from math import sqrt
import pandas as pd
from scipy.spatial.distance import pdist, squareform

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io.export import get_screenshot_as_png


output_notebook()

Loading BokehJS ...

## 4.1 Distance

### 4.1.1 What does *distance* mean?

You might well know the definition from geometry that the distance is the length of the *shortest connection between two points*.

The most fundamental assumption of geostatistics is that:

> *Observations are more similar, when they are closer together*

Thus, distance is the most important and fundamental concept of geostatistics.

### 4.1.2 Euklidean distance

We will only talk about Euklidean distance here. While most geostatistical tools just require you to calculate *distances*, it's common sense to use the Euclidean distance. But in theory geostatistics will also work on other distance frameworks.

We usually deal with **2 dimensional, cartesian** coordinates. 

The concept is generalizable to **n-dimensional** coordinates. Then, the euclidean distance is defined to be the *squareroot of the deviation squares* along each dimension.

Each observation point is represented as a *tuple* of its position in each dimension eg.:

* (2, 5) for 2D
* (2, 5, -3) for 3D
* ($x_1$, $x_2$, $\ldots$, $x_n$) for n-dimensional

For two tuples $x, y$ of the **same** dimensionality, the distance $d$:

$$ d(x, y) = \sqrt{\sum_{i=1}^n\left(x_i - y_i\right)^2}$$

* the Euklidean distance is *metric*, thus it as a zero point and you can compare distances to each other, order them and relate them to the zero point.

* the distance is **always** positive, which will be important

* a distance $d(x,y) = 0$ identifies $x$ and $y$ to be at the same **location**

### 4.1.3 Example

In the `./data/sample_positions` file, you can find the location at the the sample data shown in the last lectures were observed.

In [2]:
data = pd.read_csv('./data/sample_data.txt', sep='\s+')
coords = pd.read_csv('./data/sample_positions.txt', sep='\s+', header=None)
coords.columns = ['x', 'y']

# uncomment to look into the data
#coords.head()
#data.head()

In [3]:
# Old plot for reference
# set to True if the next cell throws errors
if False:
    obs = figure(
        title='All observations', width=600, height=600, tools=['hover'], toolbar_location='above', 
        tooltips=[('X coordinate', '@x'), ('Y coordinate', '@y'), ('Column index:', '$index')]
    )

    obs.circle(coords.x, coords.y, size=12, fill_alpha=0.6, color='#666600')
    show(obs)

In [4]:
# If you run into errors (missing PhantomJS etc) use the cell above.
imgs = []
for col in data.columns:
    f = figure(width=400, height= 170, toolbar_location=None)
    f.line(data.index, data.loc[:,col])
    
    img = get_screenshot_as_png(f)
    b = BytesIO()
    img.save(b, format='png')
    imgs.append("""<img src="data:image/png;base64,%s" >""" % base64.b64encode(b.getvalue()).decode())
    
source = ColumnDataSource({'x': coords.x, 'y': coords.y, 'imgs': imgs})
TOOLTIPS = """
<h1>Time series at #$index (@x, @y)</h1>
@imgs{safe}
"""

obs = figure(width=700, height=700, tools=['hover'], tooltips=TOOLTIPS, x_range=(0, 110), y_range=(0, 110))
obs.circle('x', 'y', source=source, size=12, fill_alpha=0.6, color='#666600')

GlyphRenderer(id='5678', ...)

In [5]:
show(obs)

Now, we can implement the distance function for two points for the 2D case only:

In [6]:
def distance(x, y):
    return sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2 )

With this, we can calculate a **distance matrix** with the index of each observation point as columns ($j$) and rows ($i$). Then a cell at $dm_{i,j}$ will contain the distance between these two points $dm_{i,j} = d(x_i, y_j)$

In [45]:
N = len(coords)
print(N)

# this creates a (N,N) matrix of -1 
dm = np.ones((N, N)) * -1

for i in range(N):
    for j in range(N):
        if i < j:
            dm[i, j] = distance(coords.loc[i], coords.loc[j])

30


In [46]:
dm.round(1)[:10, :10]

array([[-1. , 19.6, 12.2, 15.8, 66. , 74.5, 22.2, 62.1, 77.5, 35. ],
       [-1. , -1. , 15. ,  7.2, 80.8, 77. , 34.5, 65.2, 97. , 46.1],
       [-1. , -1. , -1. , 16.3, 78.2, 84.2, 34. , 71.9, 87. , 46.8],
       [-1. , -1. , -1. , -1. , 73.7, 70.7, 27.5, 58.7, 91.8, 38.9],
       [-1. , -1. , -1. , -1. , -1. , 52.3, 46.2, 47.2, 48.8, 35.7],
       [-1. , -1. , -1. , -1. , -1. , -1. , 54.3, 12.4, 99.6, 44. ],
       [-1. , -1. , -1. , -1. , -1. , -1. , -1. , 42.1, 69.5, 12.8],
       [-1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. , 92.1, 32.4],
       [-1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. , 67.1],
       [-1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. , -1. ]])

*  the diagonal is containing only `0`. This makes sense as here $i == j$ and the observation point is the same.

You can also see that there are no `-1` values left. These were added as a place holder. We know that the distance function will not return any negative value.

In [9]:
(dm < 0).any()

False

* The matrix is mirrored at the diagonal. This makes sense as: $d(x,y) == d(y,x)$

* for $N$ observation points, $\frac{N^2 - N}{2} + N$ of the $N^2$ calculations were not neccessary.

In [10]:
print('Calculations: ', N**2)
print('Unneccessary: ', int((N**2 - N) / 2 + N))

Calculations:  900
Unneccessary:  465


We can exploit that and convert the matrix to an array. That has two advantages: 

1. Many calculations in many languages are faster on 1D than on 2D arrays.
2. Code that only handles 1D arrays is easier to follow 

The only thing we have to agree on is whether we order the matrix by row or by column into the array.

In [11]:
dist = []
for i in range(N):
    for j in range(N):
        if i < j:
            dist.append(dm[i,j])
print(dist[:10])

[19.6, 12.2, 15.8, 66.0, 74.5, 22.2, 62.1, 77.5, 35.0, 30.4]


In [12]:
# We expect 900 - 465 = 435 as length
print('Calculations: ', len(dist))

Calculations:  435


In [49]:
# compare to pdist
any(pdist(coords) - np.array(dist) > 0.1)

False

We can use univariate statistics to gain some insights on the distances quite easily now. The distance data is, at the current state, univariate. Later, we will add the actual observations, to make it mulit-variate again.

In [50]:
print('Mean distance:    %.1fm' % np.mean(dist))
print('Variance:         %.1fm' % np.var(dist))
print('Std. deviation:   %.1fm' % np.std(dist))
print('Minimum:          %.1fm' % np.min(dist))
print('Maximum:          %.1fm' % np.max(dist))

Mean distance:    52.3m
Variance:         626.3m
Std. deviation:   25.0m
Minimum:          2.8m
Maximum:          116.7m


In [51]:
c, e = np.histogram(dist, bins=25)

dist_hist = figure(
    title='Point distance distribution', tooltips=[('Distance','@right m - @left m'),('Count:', '@top')],
    width=600, height=600, toolbar_location='above', tools=['hover']
)
dist_hist.quad(top=c, bottom=0, left=e[1:], right=e[:-1], fill_color='navy', line_color='white', fill_alpha=.7)

GlyphRenderer(id='6197', ...)

In [52]:
show(dist_hist)

## 4.2 Handling distance data

Calculating distances is the only the first step. The *geo* in geostatistics does not mean statistics of distances, but rather statistics **based**, or **related to**, distance.

Right now, we can't relate a specific distance to the coordinate pairs forming that distance. Thus, we need a technique to relate them back.

* Reverse-engineer the current index

* Align a mesh-grid to the distance array

* Transform between matrix and array form

#### Reverse-engineer the current index

In a distance matrix of shape $(N,N)$, the current matrix position $(i,j)$ can be calculated from the current index like:

$$ i = integer\left(\frac{index}{N}\right) $$
$$ j = index - integer\left(\frac{index}{N}\right) * N$$

#### Align a mesh grid

A *Mesh Grid*, is a matrix, that contains the current index positions as values. I.e. the value at $row=4, column=3$ would contain the tuple $(4,3)$, if we stick to a row-wise order.

In [55]:
index = []

for i in range(N):
    for j in range(N):
        if i < j:
            index.append( (i,j,) )  # the second comma indicates the tuple in Python

print(index[:10])

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10)]


_In other languages, you will have to align **1** to N (R, Matlab). I.e. Python, Java or C use **0** to N - 1._<br>
_Additionally, Matlab orderes matrices with the row index first. Python and R can change the order._

Alternatively, it is also common to create two arrays. One for each direction. You would go with a `iindex` and a `jindex` then. The difference is that these two arrays would have a more straightforward data type, as they are just integer arrays. They take only little space and access is fast. (Technically, they are lists and not arrays, but we could also create arrays)

Our `index`, however, is  a list of tuples. As the tuple can have arbitrary lengths, the computer needs to keep track of the objects in the tuples. That is quite slow. This does not only apply to Python, but also to other languages.

In [18]:
print('Point at index %d: (%.2f, %.2f)' % (13, coords.loc[13, 'x'], coords.loc[13,'y']))
print('Point at index  %d: (%.2f, %.2f)' % (7, coords.loc[7, 'x'], coords.loc[7,'y']))
print('Distance from distance matrix (2D): %.2f m' % (dm[13, 7]))

# Get distance
searched_index = -1
for i in range(len(index)):
    if index[i][0] == 7 and index[i][1] == 13:
        searched_index = i
print('Distance from distance array  (1D): %.2f m' % (dist[searched_index]))

Point at index 13: (85.00, 45.00)
Point at index  7: (38.00, 18.00)
Distance from distance matrix (2D): 54.20 m
Distance from distance array  (1D): 54.20 m


In the example above, you can see, that the if-clause changed the order, to check for the `index==7` in the first element and and `index==13` in the second element. If you switch the numbers, you won't find the correct index. Why?


You have to ask for the *smaller* point index first, because this is how we aligned the distance array `dist`, where we placed the if-clause like:

```python
for i in range(N):
    for j in range(N):
        if i < j:
            ...
```

#### Transform between matrix and array form

The last option is to write a routine, that can transform a distance matrix to a disance array.

* to apply a function or calculation to **all** values, use the **array**
* to search specific **point distances**, transform to **distance matrix**

Keep in mind that the presented code is rather slow as it is for demonstration purposes only. While you actually can use it on a few dozens of points, the distance matrix gets too large for a few hundred points and reordering gets slow

To transform from matrix to array: You can use the for loops that created the `dist` array.

To transform from array to matrix: You need the `index` array and then iterate over `dist`, like:

```python            
for idx, position in enumerate(dist):
    # this just assigns the two indices
    i,j = position 
    
    # place the distance into the upper and lower triangle
    dm[i,j] = dm[j,i] = dist[idx]

# Fill diagonals
for i in range(N):
    for j in range(N):
        if i==j:
            dm[i,j] = 0      # this is a diagonal position
```

In Python, you can find two functions in the module for scientific programming:

In [19]:
from scipy.spatial.distance import pdist, squareform

In Matlab, you will find functions of exactly the same name: https://www.mathworks.com/help/stats/pdist.html

In Octave, you'll have to load the `statistics`package first:

```Octave
%pkg install -forge statistics
%pkg load statistics
```

In all three implementations, `pdist` will return the 1D distance array, and `squareform` will transform *into both directions*

In [20]:
dist_array = pdist(coords[['x', 'y']])
distance_matrix = squareform(dist_array)

In [21]:
# check against our custom dist
(dist_array - dist < 0.1).all()

True

In [22]:
# check against our custom matrix
(distance_matrix - dm < 0.1).all()

True

In [23]:
# check back-transform
(dist_array == squareform(distance_matrix)).all()

True

Note that we need to check the *difference* for our custom calculated structures, as the calculation is implemented differently in the `scipy` module. This leads to small rounding and calculation differences and a check if the values are **exactly the same**, would fail.

For the last check, we can check for **exact match**, as `squareform` is just re-ordering the array into the matrix and vice versa. Thus, nothing is calculated and the value are the same.

## 4.3 Combining distance and observations

Finally, we are prepared to compare the distance between observations to their distances. As we have calculated the *distance* between any combination of points, we can also calculate the **difference** in value. This works in exactly the same way.

If we would use the **absolute difference**, we would actually also use an Euklidean distance of 1D coordinates.

Let's have a look at the data and the coordinates again:

In [24]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.203508,-0.164411,-0.696673,-0.555673,1.286489,-0.916081,0.572388,-0.912941,-2.571167,0.763894,...,-0.549597,0.292536,0.793382,-0.216959,1.840647,-0.582903,0.010175,-0.474006,-0.331282,-0.230938
1,-0.585733,0.035986,-0.822840,0.105027,0.806096,-1.280912,0.808606,-0.971974,-2.160372,0.733315,...,-0.593188,1.084725,0.558109,-0.216897,1.355106,-0.623354,-0.065394,-0.573369,-0.679012,0.065802
2,-0.346062,-0.188272,0.217902,0.357807,0.985086,-1.709977,-0.167685,-0.921627,-2.448770,1.115545,...,-0.356511,0.828866,1.440411,-0.151955,0.584490,-0.990339,-0.759851,-0.300697,-0.474233,0.073369
3,-0.478679,0.365037,-0.416354,0.334278,1.457831,-0.882513,0.056564,-0.646450,-2.482363,0.163104,...,-0.186352,1.360957,0.375749,-0.159243,1.155086,-0.250776,-0.036381,0.079864,0.118114,-0.364431
4,-0.736619,-0.028943,-0.418246,0.029980,0.101014,-0.959323,0.189186,-0.590574,-2.206486,1.310469,...,-0.117542,0.969032,0.379265,0.330215,1.372918,-0.732134,-0.520751,-0.340854,-0.259725,-0.327924


In [25]:
coords.head()

,x,y
0,22,78
1,3,73
2,12,85
3,9,69
4,78,43


The position index in `coords` aligns with the **column name** in `data`. The row index in `data`, was a time step. 

Now we will compose a new data structure, that adds the observations from the first time step to the coordinates

In [26]:
sample = coords.copy()
sample['z'] = data.loc[0,:].values
sample.head()

,x,y,z
0,22,78,-0.203508
1,3,73,-0.164411
2,12,85,-0.696673
3,9,69,-0.555673
4,78,43,1.286489


In [27]:
diff = []

# usually we would calculate the differences and distance in one step
for i in range(N):
    for j in range(N):
        if i < j:
            diff.append(abs(sample.z[i] - sample.z[j]))
            
print(diff[:10])

[0.03909755797114811, 0.49316483122430915, 0.35216481356446544, 1.4899973286203054, 0.7125721677891788, 0.7758968727022497, 0.7094330414195147, 2.367658970834022, 0.9674024454543844, 0.049891991094701715]


In [28]:
dist_diff = figure(
    title='Pairwise distance ~ difference', width=700, height=400,
    tooltips=[('Distance', '@x'), ('Difference', '@y')], 
    tools=['hover'], toolbar_location='above',
    x_axis_label='Distance', y_axis_label='Absoulte value difference'
)

dist_diff.circle(dist, diff, size=5, color='#336666')

GlyphRenderer(id='5853', ...)

In [29]:
show(dist_diff)

Keep in mind, that we want to analyze (and ultimately model) *statistical* spatial dependency. Thus, a single point in the graph above is not of importance. Much more, we need means to analyze their compound, or *aggregated*, behavior related to an *aggregated* distance. The aim of the model is not to precisely predict each point, but to model *average* dependence. In a second step, we can then use a technique to make value estimations.

* We find many different distance values

* Some distances are very close

* The absolute values difference is scattering to some extent

* This spread, or **variance** seems to be increasing with distance

## 4.4 Aggregation over distance

In the last section, you could see that we had a lot of point pairs at very *similar* distances. Now, we will group these point pairs together and treat them *as if* they were at the same distance.

This procedure is called **binning**, as we group the distances into *bins*. Then, as we are not using the actual pair-wise distances anymore, we refer to the independent variable as **lag**.

Setting meaningful bin edges is a main challenge in geostatistics.

Similar to binning in histograms, we want to use as **many** bins as possible, but as **little** as neccessary.

* the more bins, the more and better resolved spatial model we obtain
* at the same time, the point pairs per bin decrease and become statistically more instable

#### Defining bins

The most common approach is to form $k$ bins from $0$ to a maxium lag $l$ of same width. For $l$ various values can be chosen, either a property of the study area, like average hillslope lenghts, or a statistical proeprty of the sample like the median of all distances.

To handle the data, we create another array, storing the **higher** bounds of all bins.

In [30]:
k = 6
higher_bound = np.median(dist)
step = higher_bound / k
print('Forming %d bins from 0 to %.2f m of %.2f width' % (k, higher_bound, step))

bins = []
for ik in range(k):
    if ik == 0:
        bins.append(step)
    else:
        bins.append(bins[ik - 1] + step)
print(bins)

Forming 6 bins from 0 to 51.90 m of 8.65 width
[8.65, 17.3, 25.950000000000003, 34.6, 43.25, 51.9]


In almost all programming languages you will find pre-defined functions that can create such systematic sequences for you. In Python and Matlab this funciton has, again, the same name: `linspace`.

In [31]:
# you have to create k+1 bins and omit the first, as the start point is always included
np.linspace(0, higher_bound, k + 1)[1:]

array([ 8.65, 17.3 , 25.95, 34.6 , 43.25, 51.9 ])

It is important to always check how the point pairs are distributed over the bins. For this to happen, we have to sort the point pairs into the bins. I usually go with another array, that has the same shape as the `dist` and `diff` arrays and contains an increasing number to which bin the respective entry belongs. An entry of `3` would i.e. belong to the third (remind the `0` index) bin, from `25.96` to `34.6` distance units. 

First, we define a routine that will return the correct group.

In [32]:
def get_group(d):
    g = 0
    for b in bins:
        if d > bins[g]:
            g += 1
    return g if g < 6 else -1

Next, we will simply map it on the distances:

The Python `map` is a very effective way to apply a function to each element of an array. This is one of the main reasons, why we want the distances to be in a 1D array structure.

In Matlab `map`ping is not necessary, you can simply use a loop as these rely on compiled language elements.

In [33]:
groups = list(map(get_group, dist))
print(groups[:10])

[2, 1, 1, -1, -1, 2, -1, -1, 4, 3]


In [34]:
# to verify -> extract all the distances in group 3 
print('>=', bins[2], '   <', bins[3])
print(dm[np.where(squareform(groups) == 3)])

>= 25.950000000000003    < 34.6
[30.4 34.5 34.1 33.7 34.  27.5 31.  28.1 29.  32.4 28.2 29.5 34.5 34.
 27.5 28.6 31.  26.4 34.2 30.9 32.  31.1 32.4 26.9 32.4 34.2 28.4 34.4
 33.4 34.5 30.5 30.4 28.6 34.2 32.  30.7 33.8 32.4 32.  30.1 30.1 30.5
 28.9 28.2 31.  28.4 30.5 30.7 34.1 31.  33.5 26.2 26.4 33.5 26.3 31.
 34.2 30.7 33.8 26.2 29.5 28.9 34.4 33.4 27.9 31.8 30.9 28.1 26.3 27.9
 28.8 26.9 33.7 29.  32.  34.5 31.  31.8 31.1 30.5 30.7 28.8]


To better illustrate this, we can plot the values into a scatter plot. Here, we plot each point on one axis and all other points that contain an entry of `3` in the sqaureform of the `groups` array on the other axis. This will illustrate how the points *correlate* at the given lag bin of `25.95 < bin < 34.6`

In [35]:
searched_bin = 3
g = squareform(groups)
point_values = []
lag_values = []

for i,row in enumerate(g):
    for j,cell in enumerate(row):
        if i < j and cell == searched_bin: 
            point_values.append(sample.loc[i, 'z'])
            lag_values.append(sample.loc[j, 'z'])
            
print(len(point_values), len(lag_values))

41 41


In [36]:
crossplot = figure(
    title='Crossplot for %.1f < h < %.1f' % (bins[2], bins[3]), tools=['hover'], 
    tooltips=[('u  ', '@x'), ('u+h', '@y')], 
    x_axis_label='observation value at u', y_axis_label='observation value at u+h',
    x_range=(-1, 1.9), y_range=(-1, 1.9)
)
crossplot.circle(point_values, lag_values, size=12, fill_alpha=0.7,fill_color='#660000', line_color='white')
crossplot.ray([-1.2], [-1.2], length=6, angle=np.pi/4, line_dash='dashed', line_color='black')

GlyphRenderer(id='5953', ...)

In [37]:
show(crossplot)

## 4.5 Covariance

In the univariate statistics lecture the covariance was repeated. With the crossplot in the last section, we already illustrated, how we can split our observations into two **spatially separated** groups in a meaningful way. 

The next step is to analyze how the **similarity** between these two groups is **changing** with distance and voilá: **geostatistics!**

Recall, that calculating the covariance involved several steps. We can implement a routine to repeatedly call it. 

Note that there are pre-defined routines in Python and Matlab called `cov`.

In [38]:
def covariance(p1, p2):
    comp = [_x * _y for _x, _y in zip(p1, p2)]
    mp1 = sum(p1) / len(p1)
    mp2 = sum(p2) / len(p2)
    return sum(comp) / len(comp) - mp1 * mp2

We can directly apply this to the bin `3`, we looked at so far:

In [39]:
print(covariance(point_values, lag_values))

0.027526282574586985


Finally, we iterate over all `k=6` bins to get the covariance of all bins:

In [40]:
# do not change the k here, because we need to re-calculate bins
# and therefore also groups then!
g = squareform(groups)
covs = []


for searched_bin in range(k):

    point_values = []
    lag_values = []

    for i,row in enumerate(g):
        for j,cell in enumerate(row):
            if i < j and cell == searched_bin: 
                point_values.append(sample.loc[i, 'z'])
                lag_values.append(sample.loc[j, 'z'])
            
    covs.append(covariance(point_values, lag_values))
    
print([round(c, 2) for c in covs]) # just round for display

[0.18, 0.23, 0.48, 0.03, -0.12, -0.22]


Finally, we can plot these values against the lag. For this plot we use the upper bound of the lag class. You could also calculate the bin midpoints first.

In [41]:
covariogram = figure(
    title='Covariance plot', width=700, height=400, toolbar_location=None,
    x_axis_label='lag [m]', y_axis_label='covariance'
)
covariogram.circle(bins, covs, size=8)

GlyphRenderer(id='6079', ...)

In [42]:
show(covariogram)